## Setup

In [1]:
from kepler.pandas.menu       import ElectronSequence as Chain
from kepler.pandas.readers    import load, load_in_loop
from kepler.pandas.decorators import RingerDecorator
from typing import List
from itertools import product
import rootplotlib as rpl
import mplhep as hep
import ROOT
ROOT.gStyle.SetOptStat(0);
import numpy as np
import pandas as pd
import glob
import os
import logging
import matplotlib.pyplot as plt
plt.style.use(hep.style.ROOT)
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from copy import deepcopy
from argparse import ArgumentParser
import json
from Gaugi import GeV

from ringer.generators import ringer_generators
from ringer.utils import get_logger
from ringer.constants import DROP_COLS, L1SEEDS_PER_ENERGY, CRITERIA_CONF_NAMES, ENERGY_CHAINS, TRIG_STEPS
from ringer.data import NamedDatasetLoader

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence


## Parameters

In [2]:
HOME_PATH = os.path.expanduser('~')
et_bins = [15, 20, 30, 40, 50, 1000000]
eta_bins = [0.0, 0.8, 1.37, 1.54, 2.37, 2.50]
chain_choices = list(ENERGY_CHAINS.keys())
et_choices = list(range(len(et_bins)-1))
eta_choices = list(range(len(eta_bins)-1))
dataset = "2017_vloose"
modelpaths = [
    "/home/lucas.nunes/workspace/tunings/TrigL2_20180125_v8"
]
dev = True
cutbased= True
log = False
ets = et_choices
etas = eta_choices
chain_names = chain_choices
chain_names = [ENERGY_CHAINS[energy] for energy in chain_names]

# Initializing logger
simulation_logger = get_logger('simulate_chains', file=log)

## Building Decorators

In [3]:
simulation_logger.info('Building decorators')
decorators = list()
strategy_cols = defaultdict(list)
if cutbased:
    strategy_cols["noringer"] = list()
for modelpath, criterion in product(modelpaths, CRITERIA_CONF_NAMES.keys()):
    conf_name = CRITERIA_CONF_NAMES[criterion]
    confpath = os.path.join(modelpath, conf_name)
    env = ROOT.TEnv(confpath)
    ringer_version = env.GetValue("__version__", '')
    if ringer_version == 'should_be_filled':
        raise ValueError(f'The model from {modelpath} does not have a version. Please fill it. Version found: {ringer_version}')
    ringer_name = f'ringer_{ringer_version}'
    strat_criterion = f'{ringer_name}_{criterion}'
    simulation_logger.info(f'Building decorator for {confpath}. Version: {ringer_version}')
    decorator = RingerDecorator(strat_criterion, confpath, ringer_generators[ringer_version])
    decorators.append(decorator)
    strategy_cols[ringer_name].append(strat_criterion)
    strategy_cols[ringer_name].append(strat_criterion + '_output')

2023-02-27 13:48:17,719 - Building decorators
2023-02-27 13:48:17,730 - Building decorator for /home/lucas.nunes/workspace/tunings/TrigL2_20180125_v8/ElectronRingerTightTriggerConfig.conf. Version: v8
2023-02-27 13:48:18,213 - Building decorator for /home/lucas.nunes/workspace/tunings/TrigL2_20180125_v8/ElectronRingerMediumTriggerConfig.conf. Version: v8
2023-02-27 13:48:18,646 - Building decorator for /home/lucas.nunes/workspace/tunings/TrigL2_20180125_v8/ElectronRingerLooseTriggerConfig.conf. Version: v8
2023-02-27 13:48:19,085 - Building decorator for /home/lucas.nunes/workspace/tunings/TrigL2_20180125_v8/ElectronRingerVeryLooseTriggerConfig.conf. Version: v8


2023-02-27 13:48:17,733 | Py.RingerDecorator                      INFO Reading... /home/lucas.nunes/workspace/tunings/TrigL2_20180125_v8/ElectronRingerTightTriggerConfig.conf
2023-02-27 13:48:18,216 | Py.RingerDecorator                      INFO Reading... /home/lucas.nunes/workspace/tunings/TrigL2_20180125_v8/ElectronRingerMediumTriggerConfig.conf
2023-02-27 13:48:18,649 | Py.RingerDecorator                      INFO Reading... /home/lucas.nunes/workspace/tunings/TrigL2_20180125_v8/ElectronRingerLooseTriggerConfig.conf
2023-02-27 13:48:19,088 | Py.RingerDecorator                      INFO Reading... /home/lucas.nunes/workspace/tunings/TrigL2_20180125_v8/ElectronRingerVeryLooseTriggerConfig.conf


2023-02-27 13:48:17.745277: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-02-27 13:48:17.745347: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: UNKNOWN ERROR (-1)
2023-02-27 13:48:17.745381: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (caloba14): /proc/driver/nvidia/version does not exist
2023-02-27 13:48:17.745728: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 13:48:17.755439: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2194840000 Hz
2023-02-27 13:48:17.755564: I tensorflow/compiler/xla/service/service.c

In [4]:
strategy_cols

defaultdict(list,
            {'noringer': [],
             'ringer_v8': ['ringer_v8_tight',
              'ringer_v8_tight_output',
              'ringer_v8_medium',
              'ringer_v8_medium_output',
              'ringer_v8_loose',
              'ringer_v8_loose_output',
              'ringer_v8_vloose',
              'ringer_v8_vloose_output']})

## Building Chains

In [5]:
simulation_logger.info('Building chains')
chains = list()
step_chain_names = list()
for chain_name, strategy in product(chain_names, trigger_strategies):
    spliited_chain_name = chain_name.split('_')
    criterion = spliited_chain_name[1].replace('lh', '')
    step_chain_name = f'HLT_{chain_name.format(strategy=strategy)}'
    step_chain_names.append(step_chain_name)
    strategy_cols[strategy].append(step_chain_name)
    for trigger_step in TRIG_STEPS:
        if trigger_step != 'HLT':
            lower_chain = step_chain_name.replace('HLT', trigger_step)
            step_chain_names.append(lower_chain)
            strategy_cols[strategy].append(lower_chain)
    energy = int(spliited_chain_name[0][1:])
    l1seed = L1SEEDS_PER_ENERGY[energy]
    l2calo_column = f'{strategy}_{criterion}'
    simulation_logger.info(f'Building chain: {step_chain_name} model: {l2calo_column}')
    if strategy == 'noringer':
        chain = Chain(step_chain_name, L1Seed=l1seed)
    else:
        chain = Chain(step_chain_name, L1Seed=l1seed, l2calo_column=l2calo_column)
    chains.append(chain)

2023-02-27 13:44:44,430 - Building chains
2023-02-27 13:44:44,433 - Building chain: HLT_e24_lhtight_nod0_noringer_ivarloose model: noringer_tight
2023-02-27 13:44:44,435 - Building chain: HLT_e24_lhtight_nod0_ringer_v8_ivarloose model: ringer_v8_tight
2023-02-27 13:44:44,437 - Building chain: HLT_e26_lhtight_nod0_noringer_ivarloose model: noringer_tight
2023-02-27 13:44:44,441 - Building chain: HLT_e26_lhtight_nod0_ringer_v8_ivarloose model: ringer_v8_tight
2023-02-27 13:44:44,442 - Building chain: HLT_e60_lhmedium_nod0_noringer_L1EM24VHI model: noringer_medium
2023-02-27 13:44:44,443 - Building chain: HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI model: ringer_v8_medium
2023-02-27 13:44:44,444 - Building chain: HLT_e140_lhloose_nod0_noringer model: noringer_loose
2023-02-27 13:44:44,445 - Building chain: HLT_e140_lhloose_nod0_ringer_v8 model: ringer_v8_loose


OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'trig_L2_cl_lhtight_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (21000.0, 'ringer_v8_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 24000.0),
             ('HLT', (24000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (23000.0, 'trig_L2_cl_lhtight_et22toInf')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 26000.0),
             ('HLT', (26000.0, 'trig_EF_el_lhtight_ivarloose'))])
OrderedDict([('L1Seed', 'L1_EM22VHI'),
             ('L2Calo', (23000.0, 'ringer_v8_tight')),
             ('L2', 'trig_L2_el_cut_pt20to50'),
             ('EFCalo', 26000.0),
             ('HLT', (26000.0, 'trig_EF_el_lhtight_ivarloose'))])
Ordere

In [6]:
strategy_cols

defaultdict(list,
            {'ringer_v8': ['ringer_v8_tight',
              'ringer_v8_tight_output',
              'ringer_v8_medium',
              'ringer_v8_medium_output',
              'ringer_v8_loose',
              'ringer_v8_loose_output',
              'ringer_v8_vloose',
              'ringer_v8_vloose_output',
              'HLT_e24_lhtight_nod0_ringer_v8_ivarloose',
              'L2Calo_e24_lhtight_nod0_ringer_v8_ivarloose',
              'L2_e24_lhtight_nod0_ringer_v8_ivarloose',
              'EFCalo_e24_lhtight_nod0_ringer_v8_ivarloose',
              'HLT_e26_lhtight_nod0_ringer_v8_ivarloose',
              'L2Calo_e26_lhtight_nod0_ringer_v8_ivarloose',
              'L2_e26_lhtight_nod0_ringer_v8_ivarloose',
              'EFCalo_e26_lhtight_nod0_ringer_v8_ivarloose',
              'HLT_e60_lhmedium_nod0_ringer_v8_L1EM24VHI',
              'L2Calo_e60_lhmedium_nod0_ringer_v8_L1EM24VHI',
              'L2_e60_lhmedium_nod0_ringer_v8_L1EM24VHI',
              'EFCal

## Simulating

In [ ]:
dataset_loader = NamedDatasetLoader(dataset)
data_df_dtypes = dataset_loader.get_data_df_dtypes()
data_cols = data_df_dtypes.index
load_cols = [col for col in data_cols if col not in DROP_COLS]
last_strat = None
ibins = product([4], [4]) if dev else product(ets, etas)
last_bin = 0 if dev else len(ets)*len(etas) - 1
for i, (et_bin_idx, eta_bin_idx) in enumerate(ibins):
    start_msg = f'et {et_bin_idx} eta {eta_bin_idx}: '
    simulation_logger.info(start_msg + "Loading data_df")
    data = dataset_loader.load_data_df(load_cols, et_bin_idx, eta_bin_idx)
    simulation_logger.info(start_msg + 'Simulating')
    for decorator in decorators:
        decorator.apply(data)
    for chain in chains:
        chain.apply(data)

    for strategy in strategy_cols.keys():
        # Saves the id for future joining if necessary
        selection_cols = ['id', "region_id"] + strategy_cols[strategy]
        selected_data = data[selection_cols]
        simulation_logger.info(start_msg + f'Dumping {strategy}')
        dataset_loader.dump_strategy_df(selected_data, strategy, et_bin_idx, eta_bin_idx)
simulation_logger.info("Finished")

2023-02-27 13:11:39,090 - et 0 eta 0: Loading data_df
2023-02-27 13:11:46,036 - et 0 eta 0: Simulating


L1_EM22VHI trig_L2_cl_lhtight_et22toInf trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI trig_L2_cl_lhtight_et22toInf trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM24VHI trig_L2_cl_lhmedium_et22toInf trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI ringer_v8_medium trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI trig_L2_cl_lhloose_et22toInf trig_L2_el_cut_pt50toInf trig_EF_el_lhloose


2023-02-27 13:11:53,125 - et 0 eta 0: Dumping ringer_v8


L1_EM24VHI ringer_v8_loose trig_L2_el_cut_pt50toInf trig_EF_el_lhloose


2023-02-27 13:11:53,760 - et 0 eta 0: Dumping noringer
2023-02-27 13:11:54,052 - et 0 eta 1: Loading data_df
2023-02-27 13:11:57,747 - et 0 eta 1: Simulating


...

2023-02-27 13:13:49,103 - et 1 eta 2: Dumping noringer
2023-02-27 13:13:49,311 - et 1 eta 3: Loading data_df
2023-02-27 13:14:03,287 - et 1 eta 3: Simulating


L1_EM22VHI trig_L2_cl_lhtight_et22toInf trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI trig_L2_cl_lhtight_et22toInf trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM24VHI trig_L2_cl_lhmedium_et22toInf trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI ringer_v8_medium trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI trig_L2_cl_lhloose_et22toInf trig_L2_el_cut_pt50toInf trig_EF_el_lhloose
L1_EM24VHI ringer_v8_loose trig_L2_el_cut_pt50toInf trig_EF_el_lhloose


2023-02-27 13:14:26,228 - et 1 eta 3: Dumping ringer_v8
2023-02-27 13:14:27,991 - et 1 eta 3: Dumping noringer
2023-02-27 13:14:28,598 - et 1 eta 4: Loading data_df
2023-02-27 13:14:30,049 - et 1 eta 4: Simulating
2023-02-27 13:14:31,977 - et 1 eta 4: Dumping ringer_v8


L1_EM22VHI trig_L2_cl_lhtight_et22toInf trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI trig_L2_cl_lhtight_et22toInf trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM22VHI ringer_v8_tight trig_L2_el_cut_pt20to50 trig_EF_el_lhtight_ivarloose
L1_EM24VHI trig_L2_cl_lhmedium_et22toInf trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI ringer_v8_medium trig_L2_el_cut_pt50toInf trig_EF_el_lhmedium
L1_EM24VHI trig_L2_cl_lhloose_et22toInf trig_L2_el_cut_pt50toInf trig_EF_el_lhloose
L1_EM24VHI ringer_v8_loose trig_L2_el_cut_pt50toInf trig_EF_el_lhloose


2023-02-27 13:14:32,304 - et 1 eta 4: Dumping noringer
2023-02-27 13:14:32,427 - et 2 eta 0: Loading data_df
2023-02-27 13:15:07,915 - et 2 eta 0: Simulating
